# Reproduce Tables & Figures
_Generated: 2025-10-17_

This notebook demonstrates the full evaluation protocol on **synthetic data** (no datasets required):
1. Runs the end-to-end synthetic demo.
2. Loads outputs from `out/`.
3. Displays fusion results and bootstrap confidence intervals.


In [26]:
import os, sys, subprocess, json, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
print('Python:', sys.version)
print('CWD:', os.getcwd())


Python: 3.11.7 (main, Dec  8 2023, 18:56:58) [GCC 11.4.0]
CWD: /notebooks/MicroWildfire-repo


## 1) Run the synthetic demo
This mirrors the CLI steps in the README and writes CSV/JSON to `out/`.

In [37]:
from pathlib import Path
import zipfile, os

base = Path("/notebooks")            # adjust if your notebook cwd differs

repo = base / "MicroWildfire-repo"        # prefer your existing folder name
%cd $repo
print(os.getcwd())
!pip  install -r requirements.txt



/notebooks/MicroWildfire-repo
/notebooks/MicroWildfire-repo


In [39]:
subprocess.check_call([sys.executable, 'scripts/run_synthetic_demo.py'])
print('Done. Outputs in ./out')
import pandas as pd, json, pathlib

print("Files in out/:")
for p in pathlib.Path("out").glob("*"):
    print(" -", p.name)

# Fusion table
pd.read_csv("out/fusion_table.csv")

# Bootstrap CIs
with open("out/bootstrap_ci.json") as f:
    ci = json.load(f)
ci


Saved splits to out/splits.json
Saved fusion results to out/fusion_results.json
Saved fusion results to out/fusion_results.json
Saved bootstrap CIs to out/bootstrap_ci.json
Wrote out/fusion_table.csv
Synthetic demo complete. See out/*.csv and out/*.json
Done. Outputs in ./out
Files in out/:
 - fusion_results.json
 - splits.json
 - bootstrap_ci.json
 - fusion_table.csv


{'auroc': [0.7340338116981476, 0.7207255005084586, 0.7471056937601606],
 'brier': [0.21958011785853188, 0.21616261691376468, 0.22302971742561967],
 'ece': [0.11199672864255035, 0.0961511081031968, 0.1295454785992536],
 'eo': {'mean': 0.10787298769814647,
  'lo': 0.06720783497286847,
  'hi': 0.14983986802130705},
 'spd': {'mean': 0.08824882674844843,
  'lo': 0.05164172539399566,
  'hi': 0.13192575232663792},
 'delta_fpr': {'mean': 0.056758106541117126,
  'lo': 0.034225887420228994,
  'hi': 0.08315744576623284}}

## 2) Load fusion table

In [40]:
fusion_df = pd.read_csv('out/fusion_table.csv')
fusion_df


,Model,AUROC,Brier,ECE,Acc,F1
0,best_metrics,0.739429,0.218102,0.114077,0.653333,0.638889
1,weighted,0.721000,0.227332,0.146659,0.642000,0.616702
2,stacking,0.777309,0.187047,0.037209,0.711333,0.600185


## 3) Load bootstrap CI results and show a quick chart

In [41]:
with open('out/bootstrap_ci.json') as f:
    ci = json.load(f)
ci


{'auroc': [0.7340338116981476, 0.7207255005084586, 0.7471056937601606],
 'brier': [0.21958011785853188, 0.21616261691376468, 0.22302971742561967],
 'ece': [0.11199672864255035, 0.0961511081031968, 0.1295454785992536],
 'eo': {'mean': 0.10787298769814647,
  'lo': 0.06720783497286847,
  'hi': 0.14983986802130705},
 'spd': {'mean': 0.08824882674844843,
  'lo': 0.05164172539399566,
  'hi': 0.13192575232663792},
 'delta_fpr': {'mean': 0.056758106541117126,
  'lo': 0.034225887420228994,
  'hi': 0.08315744576623284}}

In [ ]:
vals = ci['auroc']  # [mean, lo, hi]
mean_auc, lo, hi = vals
err_lo = mean_auc - lo
err_hi = hi - mean_auc
plt.figure()
plt.errorbar([0],[mean_auc], yerr=[[err_lo],[err_hi]])
plt.title('AUROC (mean ± 95% CI)')
plt.ylabel('AUROC')
plt.xticks([])
plt.show()


In [31]:
# A) Patch scripts/run_fusion.py with _to_native and wrap all dumps
from pathlib import Path, PurePath
p = Path("scripts/run_fusion.py")
txt = p.read_text()

san = """
# --- JSON sanitizer: convert numpy dtypes/keys to native types ---
def _to_native(obj):
    import numpy as np
    if isinstance(obj, dict):
        out = {}
        for k, v in obj.items():
            if isinstance(k, np.generic):
                k = k.item()
            out[str(k)] = _to_native(v)
        return out
    elif isinstance(obj, (list, tuple)):
        return [_to_native(x) for x in obj]
    else:
        try:
            import numpy as np
            if isinstance(obj, np.generic):
                return obj.item()
        except Exception:
            pass
        return obj
"""

if "_to_native(" not in txt:
    # insert after the imports block
    txt = txt.replace("from src.metrics import compute_eo_spd_delta_fpr, auroc\n",
                      "from src.metrics import compute_eo_spd_delta_fpr, auroc\n" + san)

# wrap ALL json.dump calls
txt = txt.replace("json.dump(out, open(\"out/fusion_results.json\",\"w\"), indent=2)",
                  "json.dump(_to_native(out), open(\"out/fusion_results.json\",\"w\"), indent=2)")
txt = txt.replace("json.dump(dict(weighted=res_w, stacking=res_s, best_single=best_name, best_metrics=res_b), open(\"out/fusion_results.json\",\"w\"), indent=2)",
                  "json.dump(_to_native(dict(weighted=res_w, stacking=res_s, best_single=best_name, best_metrics=res_b)), open(\"out/fusion_results.json\",\"w\"), indent=2)")

p.write_text(txt)
print("Patched scripts/run_fusion.py")

# B) Patch src/metrics.py to string-ify per_group keys
pm = Path("src/metrics.py")
t2 = pm.read_text()
if "def compute_eo_spd_delta_fpr" in t2 and "per_group=" in t2 and "str(k)" not in t2:
    t2 = t2.replace("per_group=stats", "per_group={str(k): v for k, v in stats.items()}")
    pm.write_text(t2)
    print("Patched src/metrics.py to stringify per_group keys")



Patched scripts/run_fusion.py


In [32]:
import os, sys, subprocess
from pathlib import Path
env = os.environ.copy(); env["PYTHONPATH"] = str(Path.cwd())

proc = subprocess.run(
    [sys.executable, "-m", "scripts.run_fusion", "--config", "configs/config.yaml"],
    capture_output=True, text=True, env=env
)
print("Return code:", proc.returncode)
print("=== STDOUT ===\n", proc.stdout)
print("=== STDERR ===\n", proc.stderr)


Return code: 0
=== STDOUT ===
 Saved fusion results to out/fusion_results.json
Saved fusion results to out/fusion_results.json

=== STDERR ===
 


In [34]:
subprocess.run([sys.executable, "-m", "scripts.run_bootstrap_ci", "--config", "configs/config.yaml"],
               check=True, env=env)
subprocess.run([sys.executable, "-m", "scripts.reproduce_tables", "--config", "configs/config.yaml"],
               check=True, env=env)


Saved bootstrap CIs to out/bootstrap_ci.json
Wrote out/fusion_table.csv


CompletedProcess(args=['/usr/local/bin/python3', '-m', 'scripts.reproduce_tables', '--config', 'configs/config.yaml'], returncode=0)

In [35]:
import json, pandas as pd, pathlib
print([p.name for p in pathlib.Path("out").glob("*")])

display(pd.read_csv("out/fusion_table.csv"))
with open("out/bootstrap_ci.json") as f:
    ci = json.load(f)

# Quick glance: AUROC list and fairness dicts
print("AUROC:", ci["auroc"])
print("EO:", ci["eo"])
print("SPD:", ci["spd"])
print("ΔFPR:", ci["delta_fpr"])


['fusion_results.json', 'splits.json', 'bootstrap_ci.json', 'fusion_table.csv']


,Model,AUROC,Brier,ECE,Acc,F1
0,best_metrics,0.724097,0.223759,0.131264,0.652000,0.607519
1,weighted,0.729713,0.227162,0.173400,0.666000,0.596943
2,stacking,0.762743,0.189882,0.035124,0.704667,0.557443


AUROC: [0.7340338116981476, 0.7207255005084586, 0.7471056937601606]
EO: {'mean': 0.10787298769814647, 'lo': 0.06720783497286847, 'hi': 0.14983986802130705}
SPD: {'mean': 0.08824882674844843, 'lo': 0.05164172539399566, 'hi': 0.13192575232663792}
ΔFPR: {'mean': 0.056758106541117126, 'lo': 0.034225887420228994, 'hi': 0.08315744576623284}
